In [12]:
import tqdm
from tqdm import tqdm_notebook

import time

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as axes3d

import matplotlib.ticker as ticker

In [2]:
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import os

In [14]:
def get_df(num_models, num_exper, file_names, model_names, y_axe,  y_axe_name, title, num_show = []):
    acc_num = 3
    dist_calc_num = 7
    df = []
    for i in range(len(file_names)):
        file_name = os.path.expanduser(file_names[i])
        data = np.genfromtxt(file_name, dtype=('U10','U10','U10',float,'U10',int,'U10',int,'U10',float)).tolist()
        cur_line = -1
        for mod in range(num_models[i]):
            for j in range(num_exper[i]):
                cur_line += 1
                if y_axe == 9:
                    df.append([1.00001 - data[cur_line][acc_num],
                               1 / data[cur_line][y_axe], model_names[i][mod], title])
                else:
                    df.append([1.00001 - data[cur_line][acc_num],
                              data[cur_line][y_axe], model_names[i][mod], title])

    df = pd.DataFrame(df, columns=["Error = 1 - Recall@1", y_axe_name, "algorithm", "title"])
#     print(df.shape)
    if len(num_show) > 0:
        it = 0
        itt = 0
        num_for_iloc = []
        model_names_list = []
        for i in range(len(file_names)):
            for mod in range(len(model_names[i])):
                model_names_list.append(model_names[i][mod])
        allowed_set = set()
        same_dict = dict()
        for i in range(len(file_names)):
            for mod in range(len(model_names[i])):
                if itt in num_show:
                    allowed_set.add(model_names_list[i])
                    for j in range(num_exper[i]):
                        num_for_iloc.append(it)
                        it += 1
                else:
                    it += num_exper[i]
                itt += 1
    df = df.iloc[num_for_iloc]
    
    return df

In [15]:
def show_results(frames, title, y_axe_name, x_log=True, y_log=False,
                      dims=(18, 12), save=False, file_name='trash'):
    size = len(frames)
    ylim = [[500, 5000], [0, 1000],[0, 1000],[0, 1000]]
    a4_dims = dims
    fig, axs = plt.subplots(2, 2, figsize=a4_dims)
    for i in range(2):
        for j in range(2):
            num = i * 2 + j
            if i + j == 2:
                sns.lineplot(x="Error = 1 - Recall@1", y=y_axe_name,hue="algorithm",
                              markers=True, style="algorithm", dashes=False,
                              data=frames[num], ax=axs[i, j], linewidth=3, ms=15)
            else:
                sns.lineplot(x="Error = 1 - Recall@1", y=y_axe_name,hue="algorithm",
                              markers=True, style="algorithm", dashes=False,
                              data=frames[num], ax=axs[i, j], legend=False, linewidth=3, ms=15)
            
            axs[i, j].set_title(title[num], size='30')

            lx = axs[i, j].get_xlabel()
            ly = axs[i, j].get_ylabel()
            axs[i, j].set_xlabel(lx, fontsize=25)
            axs[i, j].set_ylabel(ly, fontsize=25)
            axs[i, j].set_xscale('log')
            if i == 1 and j == 0:
                axs[i, j].set_xticks([0.1, 1])
            else:
                axs[i, j].set_xticks([0.01, 0.1])
            axs[i, j].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
            axs[i, j].tick_params(axis='both', which='both', labelsize=25)
            axs[i, j].set_ymargin(0.075)
            if i == 0:
                axs[i, j].set_xlabel('')
            if j == 1:
                axs[i, j].set_ylabel('')
            
    plt.legend(loc=2, bbox_to_anchor=(1.05, 1, 0.5, 0.5), fontsize='30', markerscale=3, borderaxespad=0.)
#     if y_log:
#         for i in range(2):
#             for j in range(2):
#                 axs[i, j].set(yscale="log")
            
#     if x_log:
#         for i in range(2):
#             for j in range(2):
#                 axs[i, j].set(xscale="log")# num_exper = [6, 6, 3]
                
    
    if save:
        fig.savefig(file_name + ".pdf", bbox_inches='tight')

### Real Datasets

In [16]:
path = '~/Desktop/results/naive_triplet_'

In [18]:
y_axe = 9
y_axe_name = "QPS (1/s)"
model_names = [['kNN', 'HNSW', 'kNN + Kl', 'kNN + Kl + dim-red']]
num_show = [0, 1, 2, 3, 4]

num_exper = [7]
num_models = [4]
file_names = [path + 'sift.txt']
df_sift = get_df(num_models, num_exper, file_names, model_names,  y_axe, y_axe_name, title="trash", num_show=num_show)

num_exper = [5]

file_names = [path + 'gist.txt']
df_gist = get_df(num_models, num_exper, file_names, model_names, y_axe, y_axe_name, title="trash", num_show=num_show)

file_names = [path + 'glove.txt']
df_glove = get_df(num_models, num_exper, file_names, model_names, y_axe, y_axe_name, title="trash", num_show=num_show)

file_names = [path + 'deep.txt']
df_deep = get_df(num_models, num_exper, file_names, model_names, y_axe, y_axe_name, title="trash", num_show=num_show)


frames = [df_gist, df_sift, df_glove, df_deep]
show_results(frames, ['GIST', 'SIFT', 'GloVe', 'DEEP'], y_axe_name,
                  y_log=False, x_log=True, dims=(24, 14),
                  save=True, file_name='naive_real_datasets_2_2_final')
